In [33]:
import os
import sys
projectDir = '/mnt/c/Users/Julien Fournier/Documents/GitHub/RakutenTeam'
sys.path.append(projectDir)

#Define path in the config file
import src.config as config
config.path_to_project = projectDir #directory of the project
config.path_to_data = os.path.join(projectDir, 'data', 'clean') #path to the data (dataframe)
config.path_to_results = os.path.join('/mnt/g/My Drive/DST/DST-Rakuten Project', 'results') #path to where the summary of the benchmark results will be saved (csv)
config.path_to_images = '/home/jul/DST/Rakuten/Data/images/image_train_resized' #path to the folder containing images
config.path_to_models = '/mnt/c/Users/Julien Fournier/Documents/DST/RakutenProject/models' #path to the folder where the models will be saved
config.path_to_tflogs = os.path.join('/mnt/g/My Drive/DST/DST-Rakuten Project', 'tf_logs') #Path to where tensorboard logs will be saved

import pandas as pd
import numpy as np

import tensorflow as tf

from Rakuten_preprocessing import Rakuten_img_path

from src.text.classifiers import TFbertClassifier
from src.image.classifiers import ImgClassifier
from src.multimodal.classifiers import TFmultiClassifier, MetaClassifier

from src.utils.saveload import load_classifier
from src.utils.batch import fit_save_all
from src.utils.plot import plot_training_history

from sklearn.linear_model import LogisticRegression

import datetime


In [34]:
data_train = pd.read_csv(os.path.join(config.path_to_data, 'df_train_index.csv'))
data_train['testset'] = False
data_test = pd.read_csv(os.path.join(config.path_to_data, 'df_test_index.csv'))
data_test['testset'] = True
data = pd.concat([data_train, data_test], axis=0)

#merging text into token column
colnames = ['designation_translated', 'description_translated'] #['designation', 'description']#
data['tokens'] = data[colnames].apply(lambda row: ' '.join(s.lower() for s in row if isinstance(s, str)), axis=1)
    
#path to images into img_path column
data['img_path'] = Rakuten_img_path(img_folder=config.path_to_images,
                             imageid=data['imageid'], productid=data['productid'], suffix='_resized')


In [35]:
#labels of encoded classes
class_labels = data.groupby('prdtypedesignation')['prdtypeindex'].first().reset_index()
class_labels.index = class_labels['prdtypeindex']
class_labels = class_labels.drop(columns='prdtypeindex').sort_index()

## Creating train and test sets

In [36]:
Img_train = data.loc[~data['testset'], 'img_path']
Img_test = data.loc[data['testset'], 'img_path']

Txt_train = data.loc[~data['testset'], 'tokens']
Txt_test = data.loc[data['testset'], 'tokens']

y_train = data.loc[~data['testset'],'prdtypeindex']
y_test = data.loc[data['testset'],'prdtypeindex']

#To be fed into any of our sklearn classifiers, X_train and X_test
#should be dataframes with columns tokens and img_path
X_train = pd.DataFrame({'tokens': Txt_train, 'img_path': Img_train})
X_test = pd.DataFrame({'tokens': Txt_test, 'img_path': Img_test})

#All data for cross-validated scores
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)

#Number of classes
num_classes = len(np.unique(data['prdtypeindex']))

## Bert benchmarks

In [37]:
#Loading pre-trained model and specifying from_trained ansd epoch = 0 
# so that the voting classifier doesn't refit them
model_bert2 = load_classifier(name='bert-test')
model_bert2.from_trained = 'bert-test'
model_bert2.epochs = 0

model_vit2 = load_classifier(name='vit-test')
model_vit2.from_trained = 'vit-test'
model_vit2.epoch = 0

# vot_clf = MetaClassifier(base_estimators=[('bert', model_bert2), ('vit', model_vit2)], method='voting', voting='soft', weights=[0.5, 0.5])


logi_clf = LogisticRegression(C=1)
vot_clf = MetaClassifier(base_estimators=[('bert', model_bert2), ('vit', model_vit2)], final_estimator=logi_clf ,method='stacking', cv=5)

loading from Local


All model checkpoint layers were used when initializing TFCamembertModel.

All the layers of TFCamembertModel were initialized from the model checkpoint at /mnt/c/Users/Julien Fournier/Documents/DST/RakutenProject/models/base_models/camembert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


loading weights from  bert-test


/home/jul/anaconda3/envs/Rakuten/lib/python3.11/site-packages/vit_keras/utils.py:81: UserWarning:

Resizing position embeddings from 24, 24 to 14, 14



loading weights from  vit-test


In [38]:
vot_clf.fit(X_train[:1000], y_train[:1000])

loading from Local


All model checkpoint layers were used when initializing TFCamembertModel.

All the layers of TFCamembertModel were initialized from the model checkpoint at /mnt/c/Users/Julien Fournier/Documents/DST/RakutenProject/models/base_models/camembert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


loading weights from  bert-test


/home/jul/anaconda3/envs/Rakuten/lib/python3.11/site-packages/vit_keras/utils.py:81: UserWarning:

Resizing position embeddings from 24, 24 to 14, 14



loading weights from  vit-test
Found 1000 validated image filenames.


2024-03-11 00:12:42.789054: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 56.85MiB (rounded to 59610624)requested by op model_41/img_base_layers/Transformer/encoderblock_11/MultiHeadDotProductAttention_1/Softmax
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-03-11 00:12:42.789530: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-03-11 00:12:42.789581: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 548, Chunks in use: 547. 137.0KiB allocated for chunks. 136.8KiB in use in bin. 7.0KiB client-requested in use in bin.
2024-03-11 00:12:42.789604: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 24, Chunks in use: 23. 12.0KiB allocated for chunks

ResourceExhaustedError: Graph execution error:

Detected at node model_41/img_base_layers/Transformer/encoderblock_11/MultiHeadDotProductAttention_1/Softmax defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/jul/anaconda3/envs/Rakuten/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/jul/anaconda3/envs/Rakuten/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/home/jul/anaconda3/envs/Rakuten/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/home/jul/anaconda3/envs/Rakuten/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/jul/anaconda3/envs/Rakuten/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/home/jul/anaconda3/envs/Rakuten/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  

In [30]:
vot_clf.classification_score(X_train[:1000], y_train[:1000])

32/32 [==============================] - 9s 267ms/step
Found 1000 validated image filenames.
32/32 [==============================] - 8s 248ms/step


0.2920375209679759

In [5]:
#Name of the summary csv file to save results to
result_file_name = 'results_benchmark_bert.csv'

#Type of classifier
class_type = 'TFbertClassifier'

#training parameters (or list of parameters for gridsearchCV)
num_class = 27
max_length = 256
n_epochs = 2
batch_size = 32
drop_rate = 0.2
lr0 = 5e-5
lr_decay_rate = 0.8

#defining callbacks
callbacks = []
def lrscheduler(epoch, lr):
  return max(1e-6, lr * lr_decay_rate)
callbacks.append(('LearningRateScheduler', {'schedule': lrscheduler}))
callbacks.append(('EarlyStopping', {'monitor': 'val_accuracy', 'min_delta': 0, 'mode': 'max', 'patience': 2, 'restore_best_weights': True, 'verbose': 1}))

#Adding tensorboard callback as the last one
callbacks.append(('TensorBoard', {'log_dir': np.nan, 'histogram_freq': 1, 'update_freq': 'epoch'}))

#grid search number of folds
nfolds_grid = 0

#cross-validation of f1-score
nfolds_cv = 0

#network to test
base_name_list = ['camembert-base', 'camembert-base-ccnet']

#Initializing the list of parameters to batch over
params_list = []

for base_name in base_name_list:
  #Adjusting tensorboard log directory
  log_dir = os.path.join(config.path_to_tflogs, base_name, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  callbacks[-1][1]['log_dir'] = log_dir
  #adding the set of parameters to the list
  params_list.append({'modality': 'text',
                      'class': class_type,
                      'base_name': base_name,
                      'param_grid': {'max_length': max_length, 'num_class': num_class, 'drop_rate': drop_rate, 
                                    'epochs': n_epochs, 'batch_size': batch_size, 'learning_rate':lr0, 
                                    'validation_data': (X_test, y_test), 'callbacks': [callbacks]},
                      'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv
                    })
  
#Running the batch over params_list
results = fit_save_all(params_list, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, result_file_name = result_file_name)

Fitting:  camembert-base nan
{'max_length': 256, 'num_class': 27, 'drop_rate': 0.2, 'epochs': 2, 'batch_size': 32, 'learning_rate': 5e-05, 'validation_data': (                                                  tokens  \
0      bas de filles pour les enfants enfants collant...   
1      cosmic planète series en peluche avec capuche ...   
2      dolphin - robot électrique de piscine fond par...   
3                  haydaim  14/114 pokemon noir et blanc   
4      lot 4 livres partitions piano bach (busoni) - ...   
...                                                  ...   
16979  mini xf56 - gris métal mat peinture acrylique....   
16980                          fluide glacial 12 numéros   
16981             oeuvres d'histoire naturelle de goethe   
16982  pinceau de maquillage vase roses pot brosse po...   
16983                    douche solaire noir 214 cm 18 l   

                                                img_path  
0      /home/jul/DST/Rakuten/Data/images/image_train_...  
1 

2024-03-10 19:34:40.691460: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 19:34:40.691903: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 19:34:40.756817: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 19:34:40.756913: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 19:34:40.756963: I external/local_xla/xla/stream_executor

loading from Local


All model checkpoint layers were used when initializing TFCamembertModel.

All the layers of TFCamembertModel were initialized from the model checkpoint at /mnt/c/Users/Julien Fournier/Documents/DST/RakutenProject/models/base_models/camembert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-03-10 19:35:38.477660: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/2


2024-03-10 19:36:08.019990: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f645f71ef50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-10 19:36:08.020106: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2024-03-10 19:36:08.020130: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA RTX A5000, Compute Capability 8.6
2024-03-10 19:36:08.035709: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-10 19:36:08.772076: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1710095768.969156 3079263 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 230/2123 [==>...........................] - ETA: 20:10 - loss: 1.8239 - accuracy: 0.5565

## Visualize tensorflow logs in tensorboard

In [ ]:
log_path = os.path.join(config.path_to_tflogs, 'camembert-base')

# Ensure the log_path is quoted to handle spaces
quoted_log_path = f'"{log_path}"'

%reload_ext tensorboard
%tensorboard --logdir {quoted_log_path}

Reusing TensorBoard on port 6006 (pid 3002456), started 0:00:05 ago. (Use '!kill 3002456' to kill it.)

## Example usage

In [ ]:
#defining callbacks
callbacks = []
def lrscheduler(epoch, lr):
  return max(1e-6, lr * 0.8)
callbacks = callbacks.append(('LearningRateScheduler', {'schedule': lrscheduler}))
callbacks = callbacks.append(('EarlyStopping', {'monitor': 'val_accuracy', 'min_delta': 0, 'mode': 'max', 'patience': 2, 'restore_best_weights': True, 'verbose': 1, }))

clf = ImgClassifier(base_name='ResNet101', img_size=(224, 224, 3), num_class=27, drop_rate=0.2, epochs=5, batch_size=32, 
                    validation_data=(X_test, y_test), learning_rate=5e-5, callbacks=callbacks)

clf.fit(X_train, y_train)
clf.classification_score(X_test, y_test)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2024-03-10 13:07:20.850430: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.850632: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948389: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948489: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948545: I external/local_xla/xla/stream_executor

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu